In [3]:
from datasets import load_dataset

ds = load_dataset("Biddls/Onion_News")
ds['train'][0]

In [42]:
SEP = '#~#'

def get_headline(text):
    return text.split(SEP)[0].strip()

headlines = [get_headline(ds['train'][i]['text']) for i in range(10)]
headlines

['Relaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage',
 'U.S. Officials Call For Correct Amount Of Violence',
 'Kamala Harris Asks Communications Assistant If She Can Take Them Out For Coffee And Pick Their Brain Sometime',
 '25 Arrested In Fake Nursing School Diploma Scheme',
 'World’s Oldest American Dies At 72',
 'Report: Everyone Laughing At What Is A Very Silly Misunderstanding, But Don’t Be Fooled—Even Now, The Seeds Of Resentment Are Taking Root',
 'CEOs Explain How They Will Use ChatGPT',
 'FDA Moves To Ease Blood Donation Rules For Gay And Bisexual Men',
 'Study Shows Humans Still Have Genes To Grow Full Coat Of Body Hair',
 'Look What Happens When You Leave A McDonald’s Hamburger Out On A Counter For A Year']

In [8]:
import yake

kw_extractor = yake.KeywordExtractor(lan='en', n=3, top=2)

for headline in headlines[:5]:
    print(headline)
    print(kw_extractor.extract_keywords(headline))
    print()

Relaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage
[('Perfectly Happy Living', np.float64(0.0032173869679631944)), ('Relaxed Marie Kondo', np.float64(0.0035308295728302113))]

U.S. Officials Call For Correct Amount Of Violence
[('Officials Call', np.float64(0.012602360123953448)), ('Amount Of Violence', np.float64(0.012602360123953448))]

Kamala Harris Asks Communications Assistant If She Can Take Them Out For Coffee And Pick Their Brain Sometime
[('Harris Asks Communications', np.float64(0.02140921543860024)), ('Communications Assistant', np.float64(0.02140921543860024))]

25 Arrested In Fake Nursing School Diploma Scheme
[('School Diploma Scheme', np.float64(0.001881309737406442)), ('Fake Nursing School', np.float64(0.0032173869679631944))]

World’s Oldest American Dies At 72
[('Oldest American Dies', np.float64(0.0032173869679631944)), ('Oldest American', np.float64(0.02140921543860024))]



In [43]:
import spacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(headlines[0])

def extract_topics(text, max_topics=2):
    doc = nlp(text.lower()) # lower casing the text seems to result in better parsing
    noun_chunks = [chunk for chunk in doc.noun_chunks if chunk.root.pos_ != 'PRON'] # get all noun chunks, ignoring pronouns
    noun_chunks.sort(key=key, reverse=True)
    return noun_chunks

def key(span):
    # Returns the importance of a span. A span is important if it contains an entity or is long.
    entity_types = ['PERSON', 'ORG', 'GPE']
    is_entity = any(token.ent_type_ in entity_types for token in span)
    length = len(span.text.strip())
    return (is_entity, length)

for headline in headlines:
    print(extract_topics(headline))

[relaxed marie kondo, waist-high sewage]
[u.s. officials, correct amount, violence]
[kamala harris, communications assistant, their brain, coffee]
[fake nursing school diploma scheme]
[]
[a very silly misunderstanding, resentment, the seeds, report, root]
[chatgpt, ceos]
[fda, blood donation rules, gay and bisexual men]
[full coat, body hair, humans, study, genes]
[a mcdonald’s hamburger, a counter, a year]
